<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 11: NoSQL and Performance Tradeoffs** 
_For when a centralized row store just won't work_

## **Learning Objectives**
### **Theory / Be able to explain ...**
- The four concerns that drive most database implementation decisions
- The three major kinds of NoSQL models and when they are most applicable
- How each NoSQL model relates to SQL technology; how is it different and how is it the same?
- The CAP theorem and its implications for distributed databases
- What problems columnar database technology solves vis-a-vis traditional row-oriented RDBMS

### **Skills / Know how to ...**
- Represent sparse datasets as Key-Value pairs
- Handle JSON data in a relational database

--------
## **LESSON 11 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/uRoW2sojmsE" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: Physical Design Alternatives**
In this lesson we will explore various alternatives to the traditional relational DBMS. Some of the alternatives will drop SQL itself, which would seem like heresy. However, there are actually some important use cases where one might not want to use a SQL-based solution. Even when SQL is the right decision there are plenty of options to choose from that fit some use cases better than others. 

In Lesson 5 we discussed four basic design priorities:
- Minimizing Storage Space
- Maximizing Calculation Speed
- Maximizing Coherency
- Minimizing Data Corruption Risk

These tradeoffs between these priorities exist regardless of the technology used. That's why they are *design* tradeoffs. However, there is more to database systems than design. Sometimes we have to broaden our scope to consider other alternatives. 

Here are four considerations that underlie any database technology decision:
- **Flexibility / Developer Experience**  
  How easy is it for developers to learn and use the technology? It may make sense to use technology that is closer to what your programmers use everyday. 
- **Scalability / Performance Speed and Cost**  
  There is a natural tradeoff between speed and cost. Technology that works at Big Data Scale makes that tradeoff explicit. How much does each GB of storage cost? How about each GB/sec of query throughput? If we are willing to wait a little longer for each query can we save some money? 
- **Consistency / Timeliness**  
  Data is worthless if it is not available when you need it. If we don't need data to be instantaneously available, how long can we wait? If we need the data right now, then how tolerant are we of anomalies and other imperfections? If we use data replication to speed up access, do we need all copies to be 100% consistent? 
- **Technical Maturity / Technical Debt**  
  Database technology is always evolving, with new solutions coming out all the time. The newest technology may score well on the above considerations but also may come with bugs and other problems that need to be worked out. Meanwhile, older technology may be rock solid but also may limit your choices going forward. There is always the risk of being stuck with obsolete technology while your competition is beating you with something newer. 

We will start with the first two considerations, developer experience and performance, which are generally used as the rationale for NoSQL technologies that don't (necessarily) adhere to the relational database model. Then we will follow up with strategies that can further improve the speed and cost performance for any technology, providing you are able to make the right consistency and maturity tradeoffs.


---
## **NoSQL Databases**
The term "NoSQL" was first coined in the late 1990s and gained popularity among application programmers about a decade later. It refers to databases that do not rely on the relational model. That does not mean that SQL (or some close approximation) isn't used, but rather, that NoSQL systems extend beyond the traditional relational model. 

For this reason some interpret NoSQL as "Not only SQL" rather than exclusively no use of SQL at all. In fact, each of the NoSQL technologies surveyed here *could* in fact be implemented in SQL, and we will try to use relational models to explain how each relates to SQL and how it extends beyond it. 

Why would we even need to go beyond SQL? 
- **Developer Experience (DX):** As we have discussed before, there is a natural impedance mismatch between a declarative language like SQL and a more imperative application development language like Python, Java, or JavaScript. NoSQL technologies remove much of the discomfort some programmers feel when using SQL.
- **Performance:** Selectively relaxing the rules of the relational model can sometimes bring speed and cost benefits that outweigh the integrity protections of the relational model. 

For each of the models below we will discuss how it differs from the standard relational model, potential DX or performance benefits, and the most common usage scenarios. 

### **Key-Value Stores**
Key-Value (KV) stores are most useful when a table is very sparse, like this section of the NBA PlayLog data set. 

![Sparse Table](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Sparse_Table.png)

If most values in a table are blank, then why waste time and effort recording them in rows and columns. Instead, just store the data you actually have, tagged to suit however you may need to retrieve it. 

In relational terms, a KV store is just a single two column table   
`kv_store(`**`key`**`, value)`

where
- **`key`** is a unique index
- `value` is a datum to be stored

Usually the data type of the `value` is either encoded in the data itself (e.g., like i2'15' or s5'Steve`), encoded in the key, or assumed to be text. 

> **Heads Up:** We have already seen an example in Lesson 5. The Entity-Attribute-Value model can, with the right modeling conventions, be seen as a kind of KV store. The key would be a composite of the entity and attribute. 

You may be wondering how we can replace a two-dimensional table with rows and columns with a one dimensional KV store. We do it exactly like we would with composite indexes in the relational model, with the row and column encoded in the key. It's all in the patterns we use when constructing the keys.

For example, the following could be used to record the `assist`,`away`, etc. columns as key-value pairs:

| **Key** | **Value**|
| --- | ---|
| 2:away | Derrick Favors |
| 2:home | Marc Gasol |
| 18:opponent | Kyle Lowry |
| 20:num | 1 |
| 21:num | 2 |

KV Stores are commonly used for creating data caches for the web. Here, for example, is the data that Colab is keeping about *this page* (somewhat redacted) while I have it open in my web browser : 
![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Colab_Local_Storage.png)

Similar caching technology is used by web servers to minimize reads from the disk storage. If the same CSS is used on every page then why read it from disk every time? The server doesn't. Instead it serves the CSS from a highly-optimized, in-memory KV cache. 

#### **Wide-Column KV Stores**
Wide-Column data stores extend the KV model to allow data storage as multiple columns:
- The keys are just like any other KV store.
- The columns are encoded like a SQL `STRUCT` (or JSON object), with each column having a name and a value. 

The columns for each key may vary, like this:

| **Key** | **Value**|
| --- | ---|
| 2 | {away:Derrick Favors, home: Marc Gasol} |
| 18 | {opponent:Kyle Lowry} |
| 20 | {num:1} |
| 21 | {num:2} |

The effect is to condense the already very space-efficient key-value model by eliminating overlapping keys. In this case we eliminated a row of data storage by sharing the key `2` for the `away` and the `home` columns. 

#### **Summary**

- Pros
  - very fast storage and retrieval (when applicable)
  - compact storage 
  - programmer-friendly
- Cons
  - schema by convention instead of rules
  - potential for schema drift as new key types proliferate
  - no FKs or analogous way to integrate data across keys
- When to use
  - with sparse or highly volatile data where the keys need to be flexible
  - as local data stores in application development
  - when caching data to speed up application performance
- Example Products
  - [Varnish](https://varnish-cache.org/)
  - Nginx
  - Squid
  - Memcached
  - redis
  - AWS DynamoDB

### **Document Stores**
Document stores build on the KV model to construct arbitrarily complex (and data rich) databases of *semi-structured* data. Semi-structured data has a schema (so we can interpret and process it), just like any other data model, but not one we can know about in advance. Specifically, in a document store each document (roughly equivalent to a table row) can have its own schema, structuring the data however it likes. We can then only know a document schema *after* opening the document, whereas for the relational model all table schemas are known *before* doing any DML operation. 

The classic example of a document storage format is that used by Microsoft Word, which acts as a *container* for components (blocks of text, titles, images, tables, etc.). One can insert just about anything inside an MS Word file (even malware, unfortunately). MS Word will then compose and render the components as documents in real time so end users can read and edit the data contained inside. 

A more relevant example is JSON, which has become the de facto standard format for transmitting data over the web. Like with an MS Word document, a JSON object or list acts as a container, into which we can insert ... JSON objects and lists. The result is a tree structure, with objects and lists nested inside each other to some unspecified depth. 

Here, for example, is what a Colab notebook (this one, in fact) looks like when you open it in a text editor:
![Colab as JSON](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Colab_As_JSON.png)

Yes, the `ipynb` file format is really just highly structured JSON. Here's the same JSON in a nicer, pretty printed format:
![Prettified JSON](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Pretty_JSON.png)

A Colab Notebook is mostly a list of "cells" $-$ look for it in the screenshot $-$ where each cell has a `cell_type`, `metadata`, and `source`. The `source` is whatever we typed into the cell. 

We will come back to JSON in the **Pro Tips** section later in this lesson. 

#### **Summary**
- Pros
  - little or no impedance mismatch for programmers, especially when using JavaScript
  - very compact, especially for semi-structured data
- Cons
  - same as KV stores; complex queries are especially difficult
  - schema on read complicates app design and development; potentially buggy
- When to use
  - for local storage or web transmission of data
  - for complex hierarchically-structured data, where documents are composed of nested components
  - When storing "objects" in relational databases
- Example Products
  - CouchDB
  - mongoDB
  - AWS DynamoDB
  - Google Cloud Firestore

### **Graph Databases**
A graph database organizes data into three kinds of structures:
- A set of **nodes** that represent entities within the domain
- A set of **edges** (or arcs) that connect nodes to represent relationships
- **Properties** that represent the attributes of each entity or relationship

| ![Neo4J Screenshot](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Neo4J_Screenshot_Annotated.png) |
|:---:|
| Original Image Source: [*Graph Databases, Linked Data, RDF, and the Semantic Web Wasteland*]( https://medium.com/@eisenzopf/graph-databases-linked-data-rdf-and-the-semantic-web-wasteland-69e9f4347a5b) |


On a superficial level, graph databases are a lot like relational databases, with nodes = entities, properties = attributes, and edges = relationships.  A graph database, however, is much more flexible in how it defines these things: 
- Since we are not using relations, there is no need to separate the entities by type. If we like, **we can treat each node as a unique type, with its own distinct properties (kept as Key-Value pairs).** 
- Whereas a relational database treats relationships as being between entity types, graph databases define them on specific node pairs. **Any pair of nodes can be connected by an edge, regardless of type.** 
- While each node has a unique identifier, there is no concept of type-based foreign keys. Instead, **each edge is treated like an entity with a unique identifier.** The edge then has two implicit properties to reference the nodes it connects. 

With all that said, a graph database *can* be mapped into a very specific relational model where:
- There are three types of entities: nodes, edges, and properties.
- Edge entities are directional, with each having a start node and an end node.
- Properties are associated with nodes or edges as needed.

So if the graph database is really just a special case of the relational model, then what's the point? For some applications a graph database can be much, much faster than a relational database. For example, consider the database behind the [Six Degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game, where each working actor has a *Bacon Number*, BN, that represents the number of edges needed to connect them to Kevin Bacon. Kevin Bacon's BN is 0. Anyone who has appeared in a movie with him has a BN of 1. Anyone who has appeared with anyone whose BN=1 has a BN of 2, etc. The game is to guess an obscure actor's BN, with super bonus points for finding a movie actor *anywhere in the world* without a BN. 

Let all the movie credits in the world be stored in a single table called `credits`, with one row per actor and movie. To keep things as simple as possible we will assume that 
- Each actor and each movie has a unique identifier (`actor_id` and `movie_id`).
- We already know the `actor_id` identifiers for Kevin Bacon (`kevin_bacon_id`) and the other actor in question (`other_actor_id`).

Then to find all people with BN=1 we would join the credits table with itself one time:

```sql
-- Check for BN=1
SELECT distinct c1.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 USING (movie_id)
WHERE c0.actor_id = kevin_bacon_id AND c1.actor_id = other_actor_id
```
(If we want to look for a particular actor, we can just include them as `c1.actor_id` in the `WHERE` clause. If there are no results then the actor's BN is greater than 1.) 

To find all the movies that BN=1 actors appeared in would we add in another join:
```sql
SELECT distinct c2.movie_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id)
    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
WHERE c0.actor_id = kevin_bacon_id
```

We can then repeat the process with another join to get the BN=2 actors. To get the BN=3 actors we would add on two more joins to the chain. If we repeat the process out to BN=6 (the theoretical maximum for a working movie actor) then we would have 11 joins in the chain. 

```sql
SELECT distinct c11.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id) # BN=1

    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
    JOIN credits AS c3 ON (c2.movie_id = c3.movie_id) # BN=2

    JOIN credits AS c4 ON (c3.actor_id = c4.actor_id)
    JOIN credits AS c5 ON (c4.movie_id = c5.movie_id) # BN=3

    JOIN credits AS c6 ON (c5.actor_id = c6.actor_id)
    JOIN credits AS c7 ON (c6.movie_id = c7.movie_id) # BN=4

    JOIN credits AS c8 ON (c7.actor_id = c8.actor_id)
    JOIN credits AS c9 ON (c8.movie_id = c9.movie_id) # BN=5

    JOIN credits AS c10 ON (c9.actor_id = c10.actor_id)
    JOIN credits AS c11 ON (c10.movie_id = c11.movie_id) # BN=6
WHERE c0.actor_id = kevin_bacon_id AND c11.actor_id = other_actor_id
```

Running up to six queries like this seems ugly but still plenty doable, until you realize that the `credits` table might have 50 million rows. **Even with supercomputer hardware the query could take virtually forever.** With a graph database and a well-tuned search algorithm, however, we can find the specific sequence of edges connecting any actor to any other in less than a second. We can make it *even faster* if we assume that one of the actors is Kevin Bacon. 

The secret is in the specificity of the graph database model. We don't need to explore all possible actor-to-movie-to-actor connections. Instead we only need to follow the connections along the shortest path, which can be found fairly quickly via dynamic programming. If we want it even faster we can try A* search with an "explore big cast movies first" strategy. 

The achilles heel of the graph database model is bulk computation. If we need to do the same repetitive operation millions of times, perhaps concurrently, then a relational or perhaps a document database is likely a better fit, depending on the operation in question. Graph databases are designed for representation and searching, and they do those things very well, but not so much for statistical modeling or similar table-oriented applications. 

#### **Summary**
- Pros
  - flexible data model, with nodes, entities, and properties as first-class entity types
  - scales well for associative datasets with lots of relationships
- Cons
  - same as KV stores; complex queries are especially difficult 
  - while searchable, lacks most of the computational power of the relational model
  - treating each edges and property as a separate entity may not be space efficient
- When to use
  - whenever the data is naturally represented as nodes, edges, and properties
  - for network-focused apps like customer relationship management, road navigation, certain search engine operations, and file systems
  - for semantic maps (e.g., in text analytics) where the relationships represent interactions, forces, hierarchies, etc.
- Example Products
  - Neo4j
  - RedisGraph
  - AWS Neptune

  






---
## **Technology at Big Data Scale**

While any of the logical data models (relational or NoSQL) can scale up for Big Data applications, they are only partial solutions. To complete the job, we also need technology that can scale up as well. 

Absent a miraculous, once-in-every-other generation technological innovation, scaling up to Big Data requires making performance tradeoffs that deviate from the classic ACID model (Lesson 8). Here we will consider two such approaches:
- Distributed databases trade off consistency for performance and cost
- Columnar databases trade off database write and update performance for read and calculation performance. 

We will consider each in some detail below. 

### **Distributed Databases**

For most everyday business applications, a centralized database running on a server with ample data storage, a fast processor, and fast internet connectivity is more than sufficient to handle any conceivable workload. However, as an application scales up (with more users, more data, more transactions, etc.), any centralized system will begin to fail as it runs up against hard technical constraints. The telltale signs are:
- data write errors due to storage capacity limits
- queries that never finish or return errors due to processing limits
- dropped database connections due to insufficient network bandwidth

When these sorts of things happen, one can either i) work around them while waiting for bigger and better database servers or ii) try to distribute the database workload among multiple servers. 

While there are plenty of techniques that apply, two approaches prevail regardless of the technology. 

#### **Database Replication**   
A replication strategy makes redundant copies of the data, kept in separate data stores. Each copy is used independently, with local changes to any given data store propagated to every other data store (replica) behind the scenes.  

![Replication](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Replication.png)

Advantages:
- Less chance of data loss; each data store is a backup of the others
- Multiple data stores allow data to be kept geographically closer to where it is needed
- Since each data store has a full copy of the database, there are no delays due to network latency
- With fewer users per data store, there is less competition for scarce bandwidth (and fewer data delays)

Disadvantages:
- Synchronization of the replicated data among different data stores can be costly and sometimes error-prone
- Network delays in the synchronization process can cause users to get stale data from their local data store
- Increased storage costs due to data redundancy

#### **Database Partitioning**
A database partitioning strategy divides data into pieces (partitions) that are stored separately (without replication). In a dimensional data warehouse, the partitions are usually based on the dimensions rather than the facts. So, for example, if there is a customer dimension then we might choose to locate a given customer's records based on where the customer resides or who is going to serve the customer. Similarly, if newer data is used much more often than older data, then one may choose to keep the newer data on faster servers. The older data would then reside on slower, commodity data stores at a much lower price. 

![Partitioning](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Partitioning.png)

Advantages:
- Potential to locate data closer to where it is to be used
- Lower storage costs for infrequently accessed data
- No risk of stale data; there is no duplication or synchronization

Disadvantages:
- Delays and errors when queries have to assemble data from multiple partitions
- More network costs and risks, especially when queries span geographically-disparate partitions

> **Heads Up**: Replication and partitioning are somewhat complementary but not mutually exclusive. A given distributed database system may use replication for some data and partitioning for other data. The system can even use replication and partitioning for the same data if the application warrants it. We may, for example, replicate some widely-used partitions to improve query performance.

#### **The CAP Theorem**
Any distributed database solution is always a compromise, trading off among three different performance guarantees:
- **Consistency**: Every database read is 100% up-to-date (or the system throws an error)
- **Availability**: Every query receives an immediate response (even if the result is not up-to-date)
- **Partition Tolerance**: Network errors and delays do not cause the system to crash, regardless of how the data and processing are partitioned.

Unfortunately, it is not possible for a distributed database system to provide all three guarantees at once. 

![CAP Tradeoffs](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_CAP_Tradeoffs.png)

Consider, for instance, what it takes to honor the Partition Tolerance guarantee. There are really only two possible ways to handle a network error:
- Cancel all pending operations that require the network (which affects availability)
- Carry out the operations anyway with possibly stale or missing data (which affects consistency)

So, in order to guarantee that the system doesn't crash we have to allow either some data inconsistency or reduced data availability. In other words, we can have 100% consistency or 100% availability, but never both at the same time (unless we want the system to crash). The only way to satisfy all three guarantees at once is to eliminate the network (i.e., no network $\rightarrow$ no network errors or delays), reverting back to a more centralized database design.  

#### **Distributed DBMS Examples**
[Cockroach DB](https://www.cockroachlabs.com/product/) uses data replication, synchronous writes, and distributed consensus logic to ensure ACID compliance over a network. It may not always be the fastest solution $-$ ACID compliance comes at a cost $-$ but with each additional node the system gets faster without losing consistency. 

[Hadoop](https://hadoop.apache.org) uses a distributed file system ([HDFS](https://hadoop.apache.org/docs/r1.2.1/hdfs_design.html)) running on computer clusters (multiple servers in close proximity) to provide massively parallel processing power. It can then be combined with replication and partitioning to provide practically infinite storage capacity. Hadoop by itself is not a database, though it does use a kind of NoSQL Key Value Store internally. Instead it is a platform for building full-featured database systems. [Cassandra](https://cassandra.apache.org) is the most cited example of literally dozens of open source, specialized DBMS systems used for the biggest of Big Data applications. 

[Spark](https://spark.apache.org/) is an "analytics engine" that runs natively on HDFS, AWS EC2, or a variety of other platforms. While not strictly a distributed database technology, it can be used to create massive data pipelines and distributed data warehouses. 

The oldest distributed database is actually hard-wired into the Internet. The [Domain Name System](https://en.wikipedia.org/wiki/Domain_Name_System) (DNS) maps domain names (like fairfield.edu) to IP addresses (like 192.160.244.19) for devices on the internet. The system uses replication over many thousands of globally-distributed servers to keep a master list of all domain names and their IP address mappings.  

At the end of this lesson we will consider [Git](https://git-scm.com), the source code management software used by millions of programmers around the world. Git operates as a decentralized database, with each programmer working on their own "local" copies of the source code files that are kept in sync with "remote" copies being worked on by others. 

### **Columnar Databases**

Amazon Redshift, Google BigQuery, and similar cloud-native, high-performance RDBMS solutions have reclaimed some of the hype from NoSQL. Why give up relational data integrity if there is no performance benefit? Developer discomfort and impedance mismatch with relational technology is just not enough to justify the technical risks. 

So, what makes this new breed of databases worthy of the hype? It's that they are designed for dimensional data warehouses instead of transaction processing. Just like the CAP Theorem for distributed databases, we have the [RUM Conjecture](https://openproceedings.org/2016/conf/edbt/paper-12.pdf) for relational databases:

![RUM Conjecture](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_RUM_Conjecture.png)

The RUM (Read-Update-Memory) model argues that there are tradeoffs among three different kinds of performance:
- Read: How quickly can a given SELECT query execute?
- Write: How quickly can a database row be inserted or updated?
- Space: How much storage is needed to contain all the data without errors? 

Each kind of performance uses different CRUD technology, as shown on the diagram. 

#### **Row Stores vs Column Stores**

While all relational technology takes care not to use excessive storage space, there is a noticable difference between transactional databases and analytical databases.

Transactional databases work best using write-optimized **row store** technology. Data is continually being written to the database, one row at a time. In the example below each arrow represents one read or write operation.  

![Row Store](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_RowStore.png)

Because of how transaction control works, any delays in writing new rows also affect any other query that might be executing at the time. Each successive delay then takes up more and more computing capacity, causing further delays. Thus, in order to minimize the risk of system failure (dropped queries and rollbacks), it is best to prioritize writing data over reading it. 

Analytical databases tend to use read-optimized **column store** (or *columnar*) technology. Here the emphasis is on reading data organized into columns. Writing individual rows, however, can be very expensive. Thus in typical usage data rows will be added in bulk rather than one at a time. That requires just one write operation per column (shown in red below). 

![Column Store](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_ColumnStore.png)

> **Heads Up:** There is a common misperception, mostly among old-school data engineers, that column stores are non-relational. **The relational database model is about logic, not physical implementation.** Columnar databases implement all of the defining features and functions of the relational model and are thus *by definition* relational databases. The decision to store data in rows or columns is an implementation detail that has absolutely nothing to do with the relational database model.  

#### **Performance Optimizations**
While columnar databases do not excel at writing new data, they are exceptionally good at the other two dimensions of the RUM model:
- Read: Most `SELECT` queries are blazingly fast, especially if there are no joins. 
- Space: Column-wise storage allows impressive degrees of data compression that minimize storage space and speed up query performance.

**Most select queries only use a few columns of any given table.** For a dimensional data warehouse, where the fact tables tend to have many columns, that means that only a small fraction of the table needs to be processed (i.e., in memory) at a time. Let's say that we have a fact table with 50 measure (non-key) columns. If we are only using 2 columns, then the query processor has a lot less work to do in order to carry out a query. 

**A big advantage of column-wise storage is that it makes data compression really simple.** Data compression relies on exploiting repeated patterns in data. If the same pattern is repeated many times, then we can keep one copy of the pattern and then record each place where it applies. That alone can save space but we can go much further if the same pattern is repeated many times in a row. Consider, for example, the `Rating` column of our movies data. We see that 'PG-13' is repeated 27 times in a row. That means that the database only needs to store:
- the string 'PG-13'
- a run length of 26 more repeats

That is a compression ratio of about 26:1. Similar logic works for numeric data as well. Typically, numeric data tends to appear in somewhat narrow ranges, with each value in a column similar to the ones before and after it. This allows us to store the numbers as differences (above or below) some base value. Since the differences are small they can take up less storage space. For example, consider the `ShowTime` column in our example. In SQL,dates and times actually get stored as numbers (in order to simply date/tme arithmetic). However, since the show times are fairly regular, we can 
- calculate the difference between each showtime and the one above it to produce a column of mostly zeros, and then 
- compress the column even further using run lengths on the zeros

The results can be quite impressive, though not quite as good as those for columns of text data. 

**Columnar databases don't work as well with inherently row-oriented operations like table joins.** Thus, some data warehouse designs eschew dimension tables (and foreign keys) in favor of dimension columns, with a fully denormalized, one-table design. While that certainly works, it can potentially cause other problems if the source data has any anomalies. So, in recent years, columnar database solutions have implemented **optimized views that do the joins in advance**. This is done behind the scenes, in a way that is invisible to the user. For a dimensional data warehouse where the joins are the same every time anyway, such a strategy provides the speed advantage of column-wise storage and retrieval without giving up the integrity guarantees of table normalization.

#### **Columnar Database Examples**
We have already seen Google BigQuery (introduced in Lesson 3), but it is far from the only example. Others include:
- AWS Redshift
- Snowflake
- Azure SQL Data Warehouse
- Oracle Autonomous Datawarehouse 
- MariaDB ColumnStore
- PostgreSQL cstore_fdw
- Teradata
- Vertica
- Yellowbrick Data

It is also worth noting that pandas DataFrames and R Data Frames take the same column-centric approach (for the same reasons and with the same benefits/challenges) but without SQL. So, if you are comfortable using data frames for your data science projects you should be able to pick up any of these Columnar databases with only a small learning curve. 

---
## **PRO TIPS: How to Work with JSON Data in SQL**
One of the reasons why JSON is such a popular document data format is that it is so easy to **serialize** and **deserialize** JSON data for transmission over the web. Without getting into the technical details, to serialize data means that we can convert it to a possibly long string of text. Deserialization is then the conversion back from text to the original data. For exactly this sort of thing, JSON is both very fast and space efficient.

There are only three data types in JSON:
- Scalar values like strings and numbers
- Objects that bundle scalar values into named fields
- Arrays that list a sequence of scalars and/or objects in some given order

Literally everything else is represented by some combination of scalars, objects, and arrays, with nesting (arrays inside of objects inside of arrays ...) used to build up complex data structures from simpler ones. 

Since JSON data is so easy to convert to and from text strings, SQL can treat JSON as just another kind of text data. The storage and retrieval cycle looks something like this: 
1. Serialize the JSON data into a text string.
2. Store the string in the database. 
3. Retrieve the string from the database. 
4. Deserialize as JSON.

That's about as straightforward as it gets. However, what if we wanted to retrieve or modify data nested *within* a JSON string? For that we would use specialized JSON data types and functions that have been (mostly) standard SQL for a few years now. 

Most databases these days support at least one of three different ways to work with JSON data:
- Using `JSON_EXTRACT()` with JSON strings stored as regular text. The syntax varies a bit from vendor to vendor but the function signature usually looks like this:
```sql
JSON_EXTRACT(json_string, json_path)
```
where the JSON path identifies the location in the JSON string to extract the desired values.  The result is either a JSON object, JSON array, or a scalar, returned as either a JSON string ([SQLite](https://www.sqlite.org/json1.html), [AWS Redshift](https://docs.aws.amazon.com/redshift/latest/dg/json-functions.html) or a SQL equivalent ([MySQL](https://dev.mysql.com/doc/refman/8.0/en/json.html), [PostgreSQL](https://www.postgresql.org/docs/9.4/datatype-json.html), [BigQuery](https://cloud.google.com/bigquery/docs/reference/standard-sql/json_functions)). Compatible SQL data types are `STRUCT` (for JSON objects), `ARRAY` (for JSON arrays), and scalar types for numbers, strings, etc.

- Using a vendor-specific JSON data type (`JSON`, `RECORD`, `JSONB`, etc.) with associated `JSON_*()` functions (`JSON_OBJECT()`, `JSON_ARRAY()`, `JSON_TYPE()`, `JSON_SET()`, `JSON_MERGE_*()`, etc.) to create, retrieve, update, and aggregate JSON data. Like `JSON_EXTRACT()` they also use JSON paths to identify nested data. The big difference is that they can alter the data in place without extraction.

- Using the SQL standard `STRUCT` and `ARRAY` data types and functions. We explored this approach in Lesson 10. This is the approach taken by BigQuery, for example, which provides a `JSON_QUERY()` function for extracting `STRUCT` and `ARRAY` data from JSON. For complex, nested data [BigQuery uses `RECORD` schemas](https://cloud.google.com/bigquery/docs/nested-repeated) to allow query optimization and basic validation checks. If a JSON string is written to a column with a JSON RECORD schema, then BigQuery automatically casts it to fit the schema using SQL data types before storage. 

Given how ubiquitous JSON data has become, we can expect further standardization within SQL. For now, however, we will likely have to [RTFM](https://en.wikipedia.org/wiki/RTFM) the vendor's latest JSON API documentation. 













---
## **SQL AND BEYOND: Git as a Distributed DBMS**
We finish this lesson (and this course) with [Git](https://git-scm.com), a source code management system used by software developers the world over. 

### **Git and GitHub**
Successful programmers work with a lot of code. In larger projects, the source code can easily reach several million lines over thousands of files, with dozens or even hundreds of programmers all working over the same lines of code every day. If that sounds like the makings of complete chaos, then you've got the right sense of things.

One strategy used by large projects is to designate a small "core" group of people as code mavens that review every revision of the source code. Everybody else has read access to the code and has to work through a maven to make any changes. That works well, especially when the mavens coordinate well, with each specializing in a section of the code to avoid conflicts. Periodically the mavens will release the code, usually to build the next version of the working software.

The maven strategy works great up to a point and then everything breaks down. Once the core team gets bigger than a few people or the responsibility of each maven gets too big to keep in their heads, they start to make mistakes. That is exactly where Linus Torvalds, the creator and chief maven for the Linux operating system, found himself in 2005. He and his small team were reviewing hundreds of "patches" at a time for things like hardware drivers that interacted in sometimes bewildering ways. It was just too much to handle through email and programmer discipline.

So, Torvalds took a long weekend to create Git, a version control system tailored to just the kinds of problems he was facing with Linux. The name, by the way, is considered a pejorative in British slang, which is exactly what Torvalds intended. Many considered him a git, so he named it after himself.

Git works by applying a set of revision controls (i.e., the kinds of things the mavens were doing manually) to code repositories. A repository is a folder of code and other files tracked by Git. All changes made to the files are tracked continuously. Revisions to the repository files are made by committing. With each commit, the programmer includes a message describing the changes and Git checks to make sure it doesn't conflict with revisions made by other developers. If a conflict is found then the developer has to resolve the conflicts (by editing code) before being allowed to complete the commit.

[GitHub](https://github.com) is a website for hosting Git repositories. In addition to support for the usual git repository operations like cloning and committing, it also provides features like issue tracking, documentation writing (via Markdown), and forking. A clone of a repository is a local copy (i.e., in the programmers workspace) to be kept in sync with the remote copy at GitHub. As programmers push (sync) changes from their local copies to the remote GitHub, GitHub checks once again for conflicts and requires the programmer to resolve them before continuing.

Python.org maintains the Python language and tools on GitHub. Everything you could ever want to know about how Python actually works can be found there. Python is open source software, maintained by the Python.org community rather than a company. If anyone "owns" Python it is Guido van Rossum -- a long-time Google employee -- but he signed over his ownership rights to Python.org many years ago.

### **How Git Works as a Distributed DBMS**
As you are likely expecting, the lesson here is that Git uses the same replication and partitioning strategies used by more traditional distributed databases. What makes it instructive is that Git doesn't automate much of anything, instead giving programmers complete control over the synchronization and partitioning processes and giving us the opportunity to see exactly how each operation works in detail. 

#### **Git Repositories**
A Git repository is a database system for source code. It has two basic components:
- A set of working files in a **repository folder**. The files are just normal files, which the user can create, edit, and delete at will. 
- A database of **git logs** that record changes to the working files. Each log entry has a timestamped **changeset** that includes revisions to any and all working files. 

The logs go all the way back to the very beginning, allowing the working files to be recreated from scratch by "replaying" the logs as needed. We can also go backwards, reverting to any previous version. It is like Git is a "forever undo" that never forgets a logged revision. 

![Git as a DBMS](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Git_Database_System.png)

Git has a comprehensive set of repository commands for the full CRUD life cyle:
- `git init` creates a new repository from an existing file folder. Upon creating, Git adds a new subfolder called `.git` that contains the log entries and other bookkeeping. 
- `git add` queues up a file revision for adding to the logs. A revision is a record of differences (or *diffs*) between the current working file and the previous version in the logs. Multiple revisions can be added at a time. 
- `git delete` lets Git know that a file was deleted since the last log entry.
- `git status` shows what revisions have been queued and which files have unqueued revisions. 
- `git commit` makes the queued up revisions persistent as a new log entry. If no revisions have been added then git commit does nothing. 
- `git revert` undoes (soft-deletes) the most recent log entry, restoring the files to their previous state, while `git checkout` goes further back, allowing us to recreate any previous committed version. 

To delete the entire repository, just delete the working file folder. To delete the logs (and forget about Git entirely), delete the `.git` folder. 

#### **Git Replication**

One of the amazing things about Git is that it was designed from the very beginning to be distributed rather than just merely multi-user. The difference is that a multi-user database only has one copy of the data, with possibly many people executing transactions on the one copy. With git, however, each user gets their own complete copy (replica) of the working files. Users can then synchronize their copies at their leisure. 

![Git Replication](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Git_Replication.png)

Replicas are created and synchronized with the following commands:
- `git clone` makes a **local copy** of a **remote** repository. The user who creates the clone can do whatever they like to their copy without affecting anybody else. 
- `git push` publishes revisions from the local repository to a remote repository. The remote repository can refuse the revisions if they are incompatible with changes made on the remote, reporting which specific revisions are in conflict.
- `git request-pull` is a soft-push, where the remote repository indicates to the local copy that there is a revision of interest to pull.  
- `git pull` retrieves revisions from a remote, again indicating if and where there are conflicts between the two repositories so they can be resolved. 

As noted in the command summaries above, Git takes care not to force conflicting revisions in the remote to overwrite anyone's local copy. This gives each programmer ultimate control but also complicates the synchronization process quite a bit. There is actually a special kind of software engineering called DevOps that handles just this sort of thing. A lot of application programmers start out as DevOps engineers for projects led by more experienced programmers. 

#### **Git Partitioning**
Git uses branching operations to partition the repository into multiple independent copies. Each **branch** has its own versions of the working files. We can even delete a file from one branch while retaining it in another. At any given time only one branch can be active, with all repository operations (add, push, pull, etc.) only affecting the current branch. Each branch has a label (name) that we can use to refer to it as needed. The default branch (i.e., the one that exists when the repository is first created) is called 'main' (nee 'master').

![Git Partitioning](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_Git_Partitioning.png)

A few of the relevant commands:
- `git checkout` is used to switch branches, updating the working files to match. 
- `git branch` creates a new branch based on the current branch (without changing the current branch). To create a new branch *and then switch to it*, use `git checkout -b` instead. 
- `git merge` combines one branch into another, effectively closing one of the branches. 
- `git branch -d` deletes a branch entirely. 

#### **A Real Example (that we've been using all along)**

All of the course materials for this class except (except for the quizzes) are available in a remote repository at https://github.com/christopherhuntley/BUAN6510. In this case GitHub is the "remote" with a "local" working copy kept in Google Drive. 

![BUAN 6150 GitHub Repo](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_GitHub_Repo.png)

The file contents and logs (history) for each file are available by clicking on it. The branch and history are shown in red below. (Note to self: rename the branch.)
![File Browser](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_GitHub_File_Browser.png)

The history includes a line for each log entry. 
![File Browser](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_GitHub_File_History.png)

Within each log entry is a changeset, marked below in red (previous) and green (revised).

![File Revision](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_GitHub_File_Revision.png)

The local copy in Colab was synchronized with the remote copy at GitHub using (... wait for it ...) a Colab notebook with runnable form cells used for cloning, pulling, pushing, etc. 

![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L11_GitHub_Sync.png)

**With that last `git add`, `git commit`, and `git push` this course is now complete.** 



---
## **Congratulations! You've made it to the end of Lesson 11.**

There is no Lesson 12. So maybe celebrate with your beverage of choice.  



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.